In [1]:
# required libraries
! pip install qiskit
! pip install matplotlib
! pip install pylatexenc
! pip install scipy
! pip install numpy
! pip install qiskit_aer
! pip install qiskit_algorithms
! pip install qiskit_ibm_provider
! pip install qiskit-ibm-runtime
!pip install -q qiskit-machine-learning
!pip install qiskit-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=982588db604a88066f55ebe57d2a44afd1dc29be24ecfcd6b06dc0b9bd568f3b
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
# =============================================
#   MULTI-QUBIT QUANTUM COMPLEX MATRIX CALCULATOR
#   Colab Version - Interactive UI
# =============================================

!pip install qiskit ipywidgets --quiet
from IPython.display import display, Markdown
import numpy as np
import ipywidgets as widgets
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# ------------------------------
#   PARSE MATRIX
# ------------------------------
def parse_matrix(text):
    try:
        rows = text.strip().split("\n")
        matrix = [list(map(complex, r.split())) for r in rows]
        return np.array(matrix, dtype=complex)
    except:
        raise ValueError("Matrix Format Error: Enter numbers separated by spaces.")

# ------------------------------
#   MATRIX OPERATIONS
# ------------------------------
def matrix_add(A, B): return A + B
def matrix_sub(A, B): return A - B
def matrix_mul(A, B): return A @ B
def matrix_tensor(A, B): return np.kron(A, B)
def matrix_conjT(A): return np.conjugate(A.T)
def matrix_norm(A): return np.sqrt(np.sum(np.abs(A)**2))

# ------------------------------
#   QUANTUM OPERATIONS
# ------------------------------
def matrix_to_statevec(M):
    flat = M.flatten()
    mag = np.sqrt(np.sum(np.abs(flat)**2))
    if mag == 0:
        raise ValueError("Matrix cannot be zero.")
    return flat / mag

def prepare_statevector(norm_vec):
    n = int(np.log2(len(norm_vec)))
    qc = QuantumCircuit(n)
    qc.initialize(norm_vec, range(n))
    state = Statevector.from_instruction(qc)
    return qc, state

# ------------------------------
#   UI ELEMENTS
# ------------------------------
matrixA_box = widgets.Textarea(
    value="1 0\n0 1",
    description="Matrix A",
    layout=widgets.Layout(width='400px', height='150px')
)

matrixB_box = widgets.Textarea(
    value="1 0\n0 1",
    description="Matrix B",
    layout=widgets.Layout(width='400px', height='150px')
)

operations = widgets.Dropdown(
    options=["Add", "Sub", "Mul", "Tensor", "Conjugate Transpose", "Norm"],
    value="Add",
    description="Operation"
)

run_button = widgets.Button(
    description="Compute",
    button_style="success"
)

quantum_button = widgets.Button(
    description="Prepare Quantum State",
    button_style="primary"
)

output = widgets.Output()

# ------------------------------
#   EVENT HANDLERS
# ------------------------------
def on_compute_clicked(b):
    output.clear_output()
    try:
        A = parse_matrix(matrixA_box.value)
        op = operations.value

        if op in ["Add", "Sub", "Mul", "Tensor"]:
            B = parse_matrix(matrixB_box.value)

        if op == "Add":
            R = matrix_add(A, B)
        elif op == "Sub":
            R = matrix_sub(A, B)
        elif op == "Mul":
            R = matrix_mul(A, B)
        elif op == "Tensor":
            R = matrix_tensor(A, B)
        elif op == "Conjugate Transpose":
            R = matrix_conjT(A)
        elif op == "Norm":
            R = matrix_norm(A)

        with output:
            display(Markdown("### **Result:**"))
            print(R)

    except Exception as e:
        with output:
            display(Markdown(f"**Error:** {e}"))

run_button.on_click(on_compute_clicked)


def on_quantum_clicked(b):
    output.clear_output()
    try:
        A = parse_matrix(matrixA_box.value)
        norm = matrix_to_statevec(A)
        qc, sv = prepare_statevector(norm)

        with output:
            display(Markdown("### **Quantum Statevector:**"))
            print(sv)
            display(Markdown("### **Quantum Circuit:**"))
            print(qc.draw())

    except Exception as e:
        with output:
            display(Markdown(f"**Error:** {e}"))

quantum_button.on_click(on_quantum_clicked)

# ------------------------------
#   DISPLAY UI
# ------------------------------
display(Markdown("## **Multi-Qubit Quantum Complex Matrix Calculator**"))
display(widgets.HBox([matrixA_box, matrixB_box]))
display(operations)
display(run_button)
display(quantum_button)
display(output)


## **Multi-Qubit Quantum Complex Matrix Calculator**

Dropdown(description='Operation', options=('Add', 'Sub', 'Mul', 'Tensor', 'Conjugate Transpose', 'Norm'), valu…

Button(button_style='success', description='Compute', style=ButtonStyle())

Button(button_style='primary', description='Prepare Quantum State', style=ButtonStyle())

Output()

In [10]:
import matplotlib.pyplot as plt

# =============================================
#   MULTI-QUBIT QUANTUM COMPLEX MATRIX CALCULATOR
#   Enhanced UI/UX for GitHub/Colab
# =============================================

!pip install qiskit ipywidgets --quiet

from IPython.display import display, Markdown
import numpy as np
import ipywidgets as widgets
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# ------------------------------
#   MATRIX PARSING & OPERATIONS
# ------------------------------
def parse_matrix(text):
    try:
        rows = text.strip().split("\n")
        matrix = [list(map(complex, r.split())) for r in rows]
        return np.array(matrix, dtype=complex)
    except:
        raise ValueError("Matrix Format Error: Enter numbers separated by spaces.")

def matrix_add(A, B): return A + B
def matrix_sub(A, B): return A - B
def matrix_mul(A, B): return A @ B
def matrix_tensor(A, B): return np.kron(A, B)
def matrix_conjT(A): return np.conjugate(A.T)
def matrix_norm(A): return np.sqrt(np.sum(np.abs(A)**2))

def matrix_to_statevec(M):
    flat = M.flatten()
    mag = np.sqrt(np.sum(np.abs(flat)**2))
    if mag == 0:
        raise ValueError("Matrix cannot be zero.")
    return flat / mag

def prepare_statevector(norm_vec):
    n = int(np.log2(len(norm_vec)))
    qc = QuantumCircuit(n)
    qc.initialize(norm_vec, range(n))
    state = Statevector.from_instruction(qc)
    return qc, state

# ------------------------------
#   UI ELEMENTS - ATTRACTIVE
# ------------------------------
style = {'description_width': '150px'}
layout_box = widgets.Layout(width='450px', height='180px')

matrixA_box = widgets.Textarea(
    value="1 0 0\n0 1 0\n0 0 1",
    description="Matrix A:",
    layout=layout_box,
    style=style
)

matrixB_box = widgets.Textarea(
    value="1 0 0\n0 1 0\n0 0 1",
    description="Matrix B:",
    layout=layout_box,
    style=style
)

operations = widgets.Dropdown(
    options=["Add", "Sub", "Mul", "Tensor", "Conjugate Transpose", "Norm"],
    value="Add",
    description="Select Operation:",
    style=style,
    layout=widgets.Layout(width='400px')
)

run_button = widgets.Button(
    description="Compute Matrix",
    button_style="success",
    layout=widgets.Layout(width='200px', height='40px')
)

quantum_button = widgets.Button(
    description="Prepare Quantum State",
    button_style="primary",
    layout=widgets.Layout(width='250px', height='40px')
)

output = widgets.Output(layout={'border': '1px solid gray', 'padding': '10px'})

# ------------------------------
#   EVENT HANDLERS
# ------------------------------
def on_compute_clicked(b):
    output.clear_output()
    try:
        A = parse_matrix(matrixA_box.value)
        op = operations.value

        if op in ["Add", "Sub", "Mul", "Tensor"]:
            B = parse_matrix(matrixB_box.value)

        if op == "Add":
            R = matrix_add(A, B)
        elif op == "Sub":
            R = matrix_sub(A, B)
        elif op == "Mul":
            R = matrix_mul(A, B)
        elif op == "Tensor":
            R = matrix_tensor(A, B)
        elif op == "Conjugate Transpose":
            R = matrix_conjT(A)
        elif op == "Norm":
            R = matrix_norm(A)

        with output:
            display(Markdown("### ✅ **Result:**"))
            print(R)

    except Exception as e:
        with output:
            display(Markdown(f"### ❌ **Error:** {e}"))

run_button.on_click(on_compute_clicked)

def on_quantum_clicked(b):
    output.clear_output()
    try:
        A = parse_matrix(matrixA_box.value)
        norm = matrix_to_statevec(A)
        qc, sv = prepare_statevector(norm)

        with output:
            display(Markdown("### ⚛️ **Quantum Statevector:**"))
            print(sv)
            display(Markdown("### 🖥️ **Quantum Circuit:**"))
            display(qc.draw(output='mpl'))
            plt.show()

    except Exception as e:
        with output:
            display(Markdown(f"### ❌ **Error:** {e}"))

quantum_button.on_click(on_quantum_clicked)

# ------------------------------
#   DISPLAY UI
# ------------------------------
display(Markdown("## 🌌 **Multi-Qubit Quantum Complex Matrix Calculator**"))
display(Markdown("Enter complex matrices (e.g., `1+2j 0\n0 1`) and choose an operation."))
display(Markdown("**Note:** For 'Prepare Quantum State', the flattened matrix must have a length that is a power of 2 (e.g., 2, 4, 8)."))
display(widgets.HBox([matrixA_box, matrixB_box]))
display(operations)
display(widgets.HBox([run_button, quantum_button]))
display(output)


## 🌌 **Multi-Qubit Quantum Complex Matrix Calculator**

Enter complex matrices (e.g., `1+2j 0
0 1`) and choose an operation.

**Note:** For 'Prepare Quantum State', the flattened matrix must have a length that is a power of 2 (e.g., 2, 4, 8).

Dropdown(description='Select Operation:', layout=Layout(width='400px'), options=('Add', 'Sub', 'Mul', 'Tensor'…

Output(layout=Layout(border='1px solid gray', padding='10px'))